In [79]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()


1786

In [80]:
from setup import setup_src_path
print(setup_src_path())
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload



['/home/guest/Desktop/projects/third-experiments/SDA_experiments/mlm', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages', '/home/guest/Desktop/projects/third-experiments/SDA_experiments/modules', '/tmp/tmp3a_4d94j']


In [81]:
from datasets import load_from_disk,concatenate_datasets
from datasets import Dataset
import pandas as pd
from datasets import Dataset, DatasetDict


adapter_name="mlm_union_mr"

# Define paths to the CSV files
# chose any file ends with category as target
unlabeled_target_data_path = "../../datasets/sa/books_MR/target_unlabelled.csv"
dev_target_data_path = "../../datasets/sa/books_MR/dev_target.csv"
test_target_data_path = "../../datasets/sa/books_MR/test_target.csv"

# Read the CSV files into DataFrames
df_unlabeled_target = pd.read_csv(unlabeled_target_data_path)
df_dev_target = pd.read_csv(dev_target_data_path)
df_test_target = pd.read_csv(test_target_data_path)

# Convert the DataFrames to dictionaries
data_dict_unlabeled_target = df_unlabeled_target.to_dict(orient="records")
data_dict_dev_target = df_dev_target.to_dict(orient="records")
data_dict_test_target = df_test_target.to_dict(orient="records")

# Create Dataset objects from the dictionaries
dataset_unlabeled_target = Dataset.from_dict({
    "sentence": [record["sentence"] for record in data_dict_unlabeled_target],
    
})

dataset_dev_target = Dataset.from_dict({
    "sentence": [record["sentence"] for record in data_dict_dev_target],
})

dataset_test_target = Dataset.from_dict({
    "sentence": [record["sentence"] for record in data_dict_test_target],
    "label": [record["label"] for record in data_dict_test_target]
})

# Combine the unlabeled_target and dev_target datasets into a DatasetDict for training
dataset_train = DatasetDict({
    "train": dataset_unlabeled_target,
    "dev": dataset_dev_target
})

# Print the datasets to verify
print("Training Dataset with Unlabeled and Dev Splits:")
print(dataset_train)

print("\nTest Target Dataset:")
print(dataset_test_target)



Training Dataset with Unlabeled and Dev Splits:
DatasetDict({
    train: Dataset({
        features: ['sentence'],
        num_rows: 1440
    })
    dev: Dataset({
        features: ['sentence'],
        num_rows: 160
    })
})

Test Target Dataset:
Dataset({
    features: ['sentence', 'label'],
    num_rows: 400
})


In [82]:
reload(config)


train_target = dataset_train
test_target = dataset_test_target
train_target['train'][399]
 

{'sentence': '"watching harris ham it up while physically and emotionally disintegrating over the course of the movie has a certain poignancy in light of his recent death , but boyd\'s film offers little else of consequence ."'}

In [83]:
from adapters import AutoAdapterModel,init
from tqdm import tqdm

from transformers import AutoConfig, AutoModelForMaskedLM

mdlcfg = AutoConfig.from_pretrained(
    config.Config.MODEL_NAME,
 
)
model = AutoModelForMaskedLM.from_pretrained(
    config.Config.MODEL_NAME,
)
init(model)
reload(fn)
fn.print_trainable_parameters(model)

/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertFor

trainable params: 109514298 || all params: 109514298 || trainable%: 100.0


In [84]:
from adapters import ConfigUnion, LoRAConfig, PrefixTuningConfig, SeqBnConfig,SeqBnInvConfig,AdapterConfig,LoRAConfig

union_config = ConfigUnion(
    SeqBnInvConfig(reduction_factor=2),
    LoRAConfig(r=8, alpha=16),
)
model.add_adapter(adapter_name, config=union_config)

model.train_adapter([adapter_name])
model.active_adapters = adapter_name
print(model.adapter_summary())
fn.print_trainable_parameters(model)

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
mlm_union_mr             union             7,682,688       7.055       1       1
--------------------------------------------------------------------------------
Full model                               108,891,648     100.000               0
trainable params: 8274816 || all params: 117196986 || trainable%: 7.060604783812444


In [85]:
from transformers import AutoTokenizer

reload(processed)
tokenizer = AutoTokenizer.from_pretrained(config.Config.TOKENIZER_NAME)

tokenized_dataset= processed.tokenize_dataset(train_target,tokenizer)
tokenized_dataset

Map:   0%|          | 0/1440 [00:00<?, ? examples/s]

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 1440
    })
    dev: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 160
    })
})

In [86]:
dataset_test_target

Dataset({
    features: ['sentence', 'label'],
    num_rows: 400
})

In [87]:
reload(fn)
# we might not need chunks, becasue it is simple sentences
# padding tokens are going to be ignored in the collator 15%, and will be -100 in the lables
chunk_size = 128
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_dataset['train'][444:470]

results = fn.group_texts(tokenized_samples, chunk_size)
for chunk in results["labels"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

26
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'


In [88]:
# we group texts and chunk them
lm_datasets = tokenized_dataset.map(fn.group_texts, batched=True,fn_kwargs={'chunk_size': chunk_size})
len(lm_datasets['train']['labels'][1])

Map:   0%|          | 0/1440 [00:00<?, ? examples/s]

1000
440


Map:   0%|          | 0/160 [00:00<?, ? examples/s]

160


512

In [89]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
samples = [lm_datasets['train'][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")
dd=data_collator(samples)
for chunk in dd["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] [MASK] the film boasts at least a few good ideas and features some decent performances, but the result is disappointing. " [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

In [90]:

downsampled_dataset = lm_datasets

In [91]:

reload(fn)
trainer = fn.train_mlm_model(model,adapter_name,data_collator,tokenizer, downsampled_dataset['train'],downsampled_dataset['dev'])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages/accelerate/acc

In [92]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/5 [00:00<?, ?it/s]

>>> Perplexity: 13.34


In [93]:
trainer.train()

  0%|          | 0/900 [00:00<?, ?it/s]

{'loss': 2.8506, 'grad_norm': 4.929726600646973, 'learning_rate': 9e-06, 'epoch': 1.0}


  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 2.786858081817627, 'eval_runtime': 1.1456, 'eval_samples_per_second': 139.664, 'eval_steps_per_second': 4.364, 'epoch': 1.0}
{'loss': 2.703, 'grad_norm': 4.043065071105957, 'learning_rate': 1.8e-05, 'epoch': 2.0}


  0%|          | 0/5 [00:00<?, ?it/s]

...

{'eval_loss': 2.4875082969665527, 'eval_runtime': 1.1611, 'eval_samples_per_second': 137.802, 'eval_steps_per_second': 4.306, 'epoch': 5.0}


Overwriting existing adapter 'mlm_union_mr'.


{'train_runtime': 129.7371, 'train_samples_per_second': 221.987, 'train_steps_per_second': 6.937, 'train_loss': 2.6567316351996526, 'epoch': 5.0}


/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages/adapters/loading.py:165: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  st

TrainOutput(global_step=225, training_loss=2.6567316351996526, metrics={'train_runtime': 129.7371, 'train_samples_per_second': 221.987, 'train_steps_per_second': 6.937, 'train_loss': 2.6567316351996526, 'epoch': 5.0})

In [94]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/5 [00:00<?, ?it/s]

>>> Perplexity: 12.27


In [95]:
reload(processed)
testtokenized_dataset= processed.tokenize_dataset(dataset_test_target,tokenizer)
testtokenized_dataset
# we group texts and chunk them
test_dataset = testtokenized_dataset.map(fn.group_texts, batched=True,fn_kwargs={'chunk_size': chunk_size})
test_dataset


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

400


Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
    num_rows: 400
})

In [96]:
import math

# Assuming `test_dataset` is already loaded and processed
# Perform evaluation on the test dataset
test_results = trainer.evaluate(eval_dataset=test_dataset)
# Extract the test loss from the metrics
test_loss = test_results['eval_loss']

# Calculate and print perplexity based on the test loss
print(f">>> Perplexity on the test dataset: {math.exp(test_loss):.2f}")



  0%|          | 0/13 [00:00<?, ?it/s]

>>> Perplexity on the test dataset: 12.40


In [97]:
import config.config as config

trainer.model.save_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/{adapter_name}", adapter_name)


#### This is a report of mlm for goverment domain using whole genre as target and then splitt after tokenization with seed 42. I used computer metrix for preplexity and early stopping 
The results before:
>>> Perplexity: 14.10

The results after:
>>> Perplexity: 4.66